In [10]:
library("matrixStats")
library("cointReg")
library("ramify")
library("parallel")

# Include files #
source("utils.R")
source("kernel.R")
source("variance_est.R")
source("custom_statistic.R")
source("cp2010_statistic.R")

In [11]:
n<-200
rho<-0.6
sigma<-0.5
m1<-1.0
k<-as.integer(n/2) + 10
itrn1<-1

lambdas<-seq(0.1,3.0,by=0.2)
kernel<-"ba"
bandwidth<-9.0

In [12]:
# x<-ar1_data(n, k, m1, m1, rho, sigma)
# gn<-Gn(x)
# hn<-Hn_estimated(x, lambdas, kernel, bandwidth)

In [13]:
# Simulate and compute statistics #
simulate<-function(n,k,mu1,mu2,itrn1,rho,sigma){
  # One array per statistic
  g<-array(0,dim=c(itrn1,2))
  h<-array(0,dim=c(itrn1,2)) # For custom statistic

  for (i in  1 : itrn1){
    x<-gdata(n,k,mu1,mu2,rho,sigma)
    g[i,]<-gn(x,n)
    # Add new statistic here
    h[i,]<-Hn(x,n,sigma,rho)
  }
 
  list(g, h)
}


In [14]:
null_simul_hn<-function(n,k,mu1,itrn1,rho,sigma){
  q<-c(0.9,0.95,0.99)

  # One array per statistic
  h<-array(0,dim=c(itrn1,2)) # For custom statistic

  for (i in  1 : itrn1){
    x<-ar1_data(n, k, mu1, mu1, rho, sigma)
    # Add new statistic here
    h[i,]<-Hn_estimated(x, lambdas, kernel, bandwidth)
  }
  h
}

In [15]:
begin<-Sys.time()
hn_sim_stats<-null_simul_hn(n, k, m1, 500, rho, sigma)
end<-Sys.time()
print(end-begin)

Time difference of 2.474131 hours


In [16]:
# saveRDS(hn_sim_stats, file = "hn_estimated_sims_500.RDS")
hn_sim_stats<-readRDS(file="hn_estimated_sims_500_n_1000_rho_6.RDS")

Warning message in gzfile(file, "rb"):
“cannot open compressed file 'hn_estimated_sims_500_n_1000_rho_6.RDS', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [7]:
parallel_simulation<-function(i) {
    hn_stats<-null_simul_hn(n, k, m1, 500, rho, sigma)
    return(hn_stats)
}

In [8]:
input_arr<-array(0, dim=c(itrn1))
for(i in 1:itrn1) {
    input_arr[i]<-i
}

In [9]:
begin<-Sys.time()
r <- mclapply(input_arr, parallel_simulation, mc.cores=6)      ## Split this job across 10 cores
end<-Sys.time()
print(end-begin)

In [ ]:
# saveRDS(hn_sim_stats, file = "hn_estimated_sims_500.RDS")
hn_sim_stats<-readRDS(file="hn_estimated_sims_500_n_1000.RDS")

In [ ]:
hist(hn_sim_stats[,2])

In [79]:
q<-c(0.9,0.95,0.99)
hcut<-quantile(hn_sim_stats[,2], probs=q)

In [80]:
gn_sim_stats_<-readRDS(file="sim_stats_null_G.RDS")
gn_sim_stats<-gn_sim_stats_[17,,]